In [1]:
import numpy as np
import pandas as pd
import cv2
import torch
from sklearn import preprocessing
import torchvision.models as models
from torchsummary import summary
import torch.nn as nn
from models import WhaleDoModel
from sklearn.model_selection import StratifiedKFold, train_test_split
from utils import *
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
import torch
import imutils
import tensorflow as tf
from dataloader import WhaleDoDataset
import pickle as pkl
from pytorch_metric_learning import losses
from torch.utils.data import DataLoader


2022-05-19 16:01:09.726117: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
config = {
    'csv_path': 'data/metadata.csv',
    'root_dir': 'data/',

    'dataset' : {
        'channels' : 4,
        'height': None,
        'width': None,
        'mean': None,
        'std': None,
    },

    'backbone' : {
        'model': 'resnet18',
        'rep_dim': 512,
        'pretrained': True,
    },

    'projector' : {
        'hidden_dim': 1024,
        'output_dim': 2
    },
    'batch_size': 4,    
    'device': 'cpu'
}

df, label_encoder = load_csv_and_parse_dataframe(config['csv_path'], root_dir=config['root_dir'])
config['dataset']['height'], config['dataset']['width'] = get_avg_height_width(df)
config['dataset']['mean'], config['dataset']['std'] = get_mean_and_std_of_dataset(df)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)#, stratify=df['whale_id']) #can't stratify if least populated class has only one member
train_df, test_df = train_df.reset_index(), test_df.reset_index()

train_data, test_data = WhaleDoDataset(train_df, config, augmentations=True), WhaleDoDataset(test_df, config, augmentations=False)

loss_func = losses.TripletMarginLoss()

train_loader = DataLoader(train_data, config['batch_size'], shuffle=True)
test_loader = DataLoader(test_data, config['batch_size'], shuffle=False)

model = WhaleDoModel(config)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

/home/ando/.pyenv/versions/3.10.0/envs/whaledoenv/lib/python3.10/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [5]:
for i, (x_batch, y_batch) in enumerate(train_loader):
    optimizer.zero_grad()
    embeddings = model(x_batch)
    loss = loss_func(embeddings, y_batch)
    loss.backward()
    optimizer.step()

    print(i)
    break

0


In [ ]:
visualize_augmentations(dataset, n_imgs=4)

In [ ]:
img, label = dataset.__getitem__(4590)

In [ ]:
img.shape